In [1]:
from google.colab import drive
import os

In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
!cp --verbose gdrive/MyDrive/yelp_task/photo_tars/ambience_photos.tar ambience_photos.tar

'gdrive/MyDrive/yelp_task/photo_tars/ambience_photos.tar' -> 'ambience_photos.tar'


In [4]:
!tar -xf ambience_photos.tar

In [5]:
!rm ambience_photos.tar

In [6]:
!pip install pytorch-lightning

     |████████████████████████████████| 808kB 8.3MB/s 
     |████████████████████████████████| 645kB 18.8MB/s 
     |████████████████████████████████| 276kB 32.8MB/s 
     |████████████████████████████████| 829kB 34.5MB/s 
     |████████████████████████████████| 112kB 53.4MB/s 
     |████████████████████████████████| 1.3MB 51.5MB/s 
     |████████████████████████████████| 296kB 54.1MB/s 
     |████████████████████████████████| 143kB 46.1MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=e05bcad5cb2b56db8c486e433f6789d54a5bda4a6396b79203984966b9709610
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [7]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-82993f7b-919c-50be-38df-8b0368873d24)


In [8]:
from __future__ import print_function
from __future__ import division

In [9]:
import torch
from torch.utils.data import DataLoader

In [10]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

In [11]:
import torchvision
from torchvision import transforms

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
import time
#import os
import copy
from tqdm.notebook import tqdm

In [14]:
from gdrive.MyDrive.yelp_task.lightning_model import LightningTransfer, init_cls_model
from gdrive.MyDrive.yelp_task.dataset_loaders import YelpDataset
from gdrive.MyDrive.yelp_task.metrics import multiclass_stats

In [15]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [16]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print("PyTorch Lightning Version: ",pl.__version__)

PyTorch Version:  1.8.1+cu101
Torchvision Version:  0.9.1+cu101
PyTorch Lightning Version:  1.3.0


In [17]:
photo_dir = 'photos/'
csv_path = "business_ambience.csv"
csv_file = pd.read_csv(csv_path)

In [18]:
assert len(os.listdir('photos')) == len(csv_file)

In [19]:
FLAGS = {
    'model_name': 'densenet',
    'num_classes': 6,
    'batch_size': 512,  # 512 / N gpu
    'num_workers': 4,  # 4 per gpu
    'learning_rate': 0.02,  # 0.02 * N gpu
    'max_epochs': 20,  # arbitrary
    'feature_extract': True,
    'use_pretrained': True,
    'multilabel': True,
    'threshold': 0.5,
    'class_weight': [3.513, 1.63, 6.084, 9.839, 6.502, 4.625]
        }

In [20]:
transfer_model, input_size = init_cls_model(FLAGS)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


In [21]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'dev': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [22]:
print("Initializing Datasets and Dataloaders...")

Initializing Datasets and Dataloaders...


In [23]:
csv_file = pd.read_csv(csv_path)

In [24]:
csv_file.head()

,photo_id,touristy,hipster,romantic,divey,intimate,upscale
0,ZlTwL6uWx6rW_L9Df5RT8A,False,False,True,False,True,False
1,fHbSMxueQfXFRb9e-6bJuw,False,False,False,True,False,False
2,74oWvVVIjms9LjfHQOgxMQ,False,False,False,False,False,True
3,QY6c1OKsIpujF4MDHQdbag,False,True,False,False,False,False
4,0AYEzNJYFF2PeXo71cpKuw,False,True,False,False,False,False


In [25]:
from sklearn.model_selection import train_test_split

In [26]:
train, test = train_test_split(csv_file, test_size=0.2, random_state=42)

In [27]:
dev, test = train_test_split(test, test_size=0.5, random_state=42)

In [28]:
data_frames = {'train': train,
              'dev': dev,
              'test': test}

In [29]:
# Create training and validation datasets
image_datasets = {x: YelpDataset(data_frames[x], photo_dir, data_transforms[x]) for x in ['train', 'dev', 'test']}
# Create training and validation dataloaders
dataloaders_dict = {x: DataLoader(image_datasets[x],
                                  batch_size=FLAGS['batch_size'],
                                  shuffle=True,
                                  num_workers=FLAGS['num_workers'],
                                  pin_memory=True) for x in ['train', 'dev']}
dataloaders_dict['test'] = DataLoader(image_datasets['test'],
                                      batch_size=FLAGS['batch_size'],
                                      shuffle=False,
                                      num_workers=FLAGS['num_workers'],
                                      pin_memory=True)

In [30]:
dataloaders_dict

{'dev': <torch.utils.data.dataloader.DataLoader at 0x7fe864014f10>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7fe864014e10>,
 'train': <torch.utils.data.dataloader.DataLoader at 0x7fe864014e90>}

In [31]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
early_stop_callback = EarlyStopping(monitor='val_loss', patience=3)

In [32]:
trainer = pl.Trainer(callbacks=[checkpoint_callback, early_stop_callback],
                     progress_bar_refresh_rate=1,
                     stochastic_weight_avg=True, precision=16, max_epochs=FLAGS['max_epochs'], gpus=-1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


In [33]:
trainer.fit(transfer_model, dataloaders_dict['train'], dataloaders_dict['dev'])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | DenseNet          | 7.0 M 
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
6.2 K     Trainable params
7.0 M     Non-trainable params
7.0 M     Total params
27.840    Total estimated model params size (MB)


In [34]:
trainer.save_checkpoint(f"gdrive/MyDrive/yelp_task/yelp_data/checkpoints/{FLAGS['model_name']}_amb.ckpt")

In [35]:
print(checkpoint_callback.best_model_path)

/content/lightning_logs/version_0/checkpoints/epoch=11-step=623.ckpt


In [36]:
transfer_model, _ = init_cls_model(FLAGS, checkpoint_callback.best_model_path)

In [37]:
trainer.test(transfer_model, dataloaders_dict['test'])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.9671331644058228}
--------------------------------------------------------------------------------


[{'test_loss': 0.9671331644058228}]

In [38]:
model = transfer_model.eval().cuda(device=0)

In [39]:
y_pred, y_true = [], []

In [40]:
for image, labels in tqdm(image_datasets['test']):
    output = model(image.unsqueeze(0).cuda(0)).cpu().data.numpy()[0]
    if output.size == 1:
        out_array = np.zeros(FLAGS['num_classes']).astype('int')
        out_array[output] = 1
        y_pred.append(out_array)
    else:
        y_pred.append(output)
    y_true.append(labels.numpy().astype('int'))

In [41]:
cls_report, stats = multiclass_stats(np.array(y_pred), np.array(y_true))
print(cls_report)

              precision    recall  f1-score   support

           0       0.63      0.38      0.48      1206
           1       0.62      0.54      0.57      1437
           2       0.63      0.23      0.34      1277
           3       0.73      0.20      0.31       996
           4       0.65      0.20      0.31      1369
           5       0.73      0.31      0.44      1337

   micro avg       0.65      0.32      0.43      7622
   macro avg       0.66      0.31      0.41      7622
weighted avg       0.66      0.32      0.41      7622
 samples avg       0.65      0.34      0.43      7622



In [42]:
stats

{'f1_score': 0.40821280701467105,
 'hamming_loss': 0.32948679016094745,
 'hamming_score': 0.33356615042008303,
 'roc_auc_score': 0.599962033598466}

In [43]:
from sklearn.metrics import multilabel_confusion_matrix

In [44]:
multilabel_confusion_matrix(y_true, y_pred)

array([[[1812,  742],
        [ 275,  464]],

       [[1373,  664],
        [ 483,  773]],

       [[1839,  981],
        [ 177,  296]],

       [[2222,  797],
        [  75,  199]],

       [[1774, 1090],
        [ 150,  279]],

       [[1801,  921],
        [ 155,  416]]])

In [45]:
thresholds = []

In [46]:
for i in [0.3, 0.4, 0.5, 0.6, 0.7]:
    FLAGS['threshold'] = i
    transfer_model, _ = init_cls_model(FLAGS, checkpoint_callback.best_model_path)
    model = transfer_model.eval().cuda(device=0)
    y_pred, y_true = [], []
    for image, labels in tqdm(image_datasets['test']):
        output = model(image.unsqueeze(0).cuda(0)).cpu().data.numpy()[0]
        if output.size == 1:
            out_array = np.zeros(FLAGS['num_classes']).astype('int')
            out_array[output] = 1
            y_pred.append(out_array)
        else:
            y_pred.append(output)
        y_true.append(labels.numpy().astype('int'))
    _, stats = multiclass_stats(np.array(y_pred), np.array(y_true))
    thresholds.append(stats)

In [48]:
thresholds

[{'f1_score': 0.3660344279781109,
  'hamming_loss': 0.5543577285150318,
  'hamming_score': 0.25460066808381415,
  'roc_auc_score': 0.5863992222529519},
 {'f1_score': 0.3906866316527526,
  'hamming_loss': 0.4359246887336775,
  'hamming_score': 0.2910061747140399,
  'roc_auc_score': 0.5895255137005644},
 {'f1_score': 0.40821280701467105,
  'hamming_loss': 0.32948679016094745,
  'hamming_score': 0.33356615042008303,
  'roc_auc_score': 0.599962033598466},
 {'f1_score': 0.3919285443102091,
  'hamming_loss': 0.25589634578398623,
  'hamming_score': 0.3238182002226946,
  'roc_auc_score': 0.6117972220960093},
 {'f1_score': 0.32640455791086803,
  'hamming_loss': 0.21110436278975606,
  'hamming_score': 0.2352464824374937,
  'roc_auc_score': 0.6292411321727281}]